In [10]:
import gurobipy as gp
from gurobipy import GRB

 # Parameters

In [11]:
arcs, costs = gp.multidict({
    ('Cleveland','Boston'): 3, 
    ('Cleveland','Chicago'): 2,
    ('Cleveland','StLouis') :7,
    ('Cleveland','Lexington'): 6,
    ('Bedford','Boston'): 7,
    ('Bedford','Chicago'): 5,
    ('Bedford','StLouis'): 2,
    ('Bedford','Lexington'): 3,
    ('York', 'Boston'): 2,
    ('York', 'Chicago'): 5,
    ('York', 'StLouis'): 4,
    ('York', 'Lexington'): 5,
})


supply = {
    'Cleveland': 5_000,
    'Bedford':  6_000,
    'York': 2_500,
}

demand = {
    'Boston': 6_000,
    'Chicago': 4_000,
    'StLouis': 2_000,
    'Lexington': 1_500,
}

# Model
## Decision Variables

In [12]:
m = gp.Model('transportation')
send = m.addVars(arcs,  vtype=GRB.CONTINUOUS, name='send')

 # Objective Function

In [13]:
Z = send.prod(costs)
m.ModelSense = GRB.MINIMIZE
m.setObjective(Z)

 # Constraints

In [14]:
source = m.addConstrs(
    (send.sum(i, '*') <= s for i,s in supply.items() ),
    name='source'
)

sink = m.addConstrs(
    (send.sum('*', j) == d for j,d in demand.items()),
    name='sink'
)

In [15]:
m.update()
print(f'Number of constraints {m.NumConstrs}')
print(f'Number of variables {m.NumVars}')
print(f'Constraints are \n {m.getConstrs()} \n')

Number of constraints 7
Number of variables 12
Constraints are 
 [<gurobi.Constr source[Cleveland,5000]>, <gurobi.Constr source[Bedford,6000]>, <gurobi.Constr source[York,2500]>, <gurobi.Constr sink[Boston,6000]>, <gurobi.Constr sink[Chicago,4000]>, <gurobi.Constr sink[StLouis,2000]>, <gurobi.Constr sink[Lexington,1500]>] 



## Optimization process

In [16]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x83eab019
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+03]
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8500000e+04   3.500000e+03   0.000000e+00      0s
       2    3.9500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.950000000e+04


## Results

In [17]:
print(f'Objective value: ${m.objVal:,.2f}\n')

Objective value: $39,500.00



In [18]:
for v in m.getVars():
    if abs(v.X) > 0.00001:
        print(f'{v.VarName}: {v.x:,.0f}') 

send[Cleveland,Boston]: 3,500
send[Cleveland,Chicago]: 1,500
send[Bedford,Chicago]: 2,500
send[Bedford,StLouis]: 2,000
send[Bedford,Lexington]: 1,500
send[York,Boston]: 2,500
